# Plots of co-channel and adjacent channel interference transforms

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    torchsig_float_data_type,
    torchsig_complex_data_type,
    low_pass,
    noise_generator
)

import numpy as np
import scipy.signal as sp
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

In [ ]:
def generate_tone_signal(num_iq_samples: int = 10, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal.

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['tone'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
# visualize default filter design
cutoff = 0.125
transition_bandwidth = 0.125
sample_rate = 1.0
lpf_weights = low_pass(cutoff=cutoff,transition_bandwidth=transition_bandwidth,sample_rate=sample_rate) # design lowpass filter

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()
spectrum, freqs, _ = ax.magnitude_spectrum(lpf_weights, Fs=sample_rate, scale='linear', sides='twosided', color='white');
ax.set_yscale('log')
ax.set_xlabel('Frequency');
ax.set_ylabel('Magnitude [log]');

In [ ]:
def cochannel_interference(
    data: np.ndarray,
    power: float = 1.0,
    filter_weights: np.ndarray = low_pass(0.125, 0.125, 1.0),
    color: str = 'white',
    continuous: bool = True,
    rng: np.random.Generator = np.random.default_rng(seed=None)
) -> np.ndarray:
    """Applies uncorrelated co-channel interference to the baseband data, modeled as shaped noise with specified parameters.

    Args:
        data (np.ndarray): Complex valued IQ data samples.
        power (float): Interference power (linear, positive). Default 1.0 W (0 dBW).
        filter_weights: Lowpass interference shaping filter weights. Default low_pass(0.125, 0.125, 2.0).
        color (str): Base noise color, supports 'white', 'pink', or 'red' noise frequency spectrum types. Default 'white'.
        continuous (bool): Sets noise to continuous (True) or impulsive (False). Default True.
        rng (np.random.Generator, optional): Random number generator. Defaults to np.random.default_rng(seed=None).
    
    Returns:
        np.ndarray: Data with added uncorrelated co-channel interference.
    
    """
    N = len(data)
    noise_samples = noise_generator(N, power, color, continuous, rng)
    shaped_noise = np.convolve(noise_samples, filter_weights)[-N:]
    
    # correct shaped noise power (do not assume filter is prescaled)
    est_power = np.sum(np.abs(shaped_noise)**2)/len(shaped_noise)
    interference = np.sqrt(power / est_power) * shaped_noise 
    return (data + interference).astype(torchsig_complex_data_type)

In [ ]:
rng = np.random.default_rng(seed=42)

N = 8192
sample_rate = 1.0
power = 0.1
tone_freq = 0.1
filter_weights = low_pass(0.125, 0.125, 1.0)
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
tone_data = tone_bb_data * np.exp(2j * np.pi * tone_freq * np.arange(N) / sample_rate) * np.sqrt(N) # f0 = 0.2, total power = 1.0 W
est_power = np.sum(np.abs(tone_data)**2)/len(tone_data)
print("tone_data power: ", est_power) # verify 1.0 W

data_co = cochannel_interference(
    data = tone_data,
    power = power,
    filter_weights = filter_weights,
    color = 'white',
    continuous = True,
)
est_power = np.sum(np.abs(data_co)**2)/len(data_co)
print("data_co power: ", est_power) # verify 1.0 W + 1.0 W = 2.0 W (uncorrelated)

# verify peaks
D = np.abs(np.fft.fft(data_co, norm='ortho'))
freqs = np.fft.fftfreq(N) * sample_rate

peaks, _ = sp.find_peaks(D, height=1.0, distance=N/2)
max_peak_ind = np.argmax(D[peaks])
#print(freqs[max_peak_ind])
print(peaks)
print(freqs[peaks])
print(D[peaks])


# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(tone_data, Fs=sample_rate, scale='linear', sides='twosided', color='white');
spectrum, freqs, _ = ax.magnitude_spectrum(data_co, Fs=sample_rate, scale='linear', sides='twosided', color='red');
ax.set_yscale('log')
ax.set_xlabel('Frequency');
ax.set_ylabel('Magnitude [log]');

In [ ]:
rng = np.random.default_rng(seed=42)

N = 16384
sample_rate = 2.0
power = 0.01
tone_freq = -0.04
filter_weights = low_pass(0.1, 0.15, 2.0)
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
tone_data = tone_bb_data * np.exp(2j * np.pi * tone_freq * np.arange(N) / sample_rate) * np.sqrt(N) # f0 = 0.2, total power = 1.0 W
est_power = np.sum(np.abs(tone_data)**2)/len(tone_data)
print("tone_data power: ", est_power) # verify 1.0 W

data_co = cochannel_interference(
    data = tone_data,
    power = power,
    filter_weights = filter_weights,
    color = 'white',
    continuous = True,
)
est_power = np.sum(np.abs(data_co)**2)/len(data_co)
print("data_co power: ", est_power) # verify 1.0 W + 1.0 W = 2.0 W (uncorrelated)

# verify peaks
D = np.abs(np.fft.fft(data_co, norm='ortho'))
freqs = np.fft.fftfreq(N) * sample_rate
peaks, _ = sp.find_peaks(D, height=1.0, distance=N/2)
max_peak_ind = np.argmax(D[peaks])
print("peak freqs:, ", freqs[peaks])

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(tone_data, Fs=sample_rate, scale='linear', sides='twosided', color='white');
spectrum, freqs, _ = ax.magnitude_spectrum(data_co, Fs=sample_rate, scale='linear', sides='twosided', color='red');
ax.set_yscale('log')
ax.set_xlabel('Frequency');
ax.set_ylabel('Magnitude [log]');

In [ ]:
def adjacent_channel_interference(
    data: np.ndarray,
    sample_rate: float = 4.0,
    power: float = 1.0,
    center_frequency: float = 0.2,
    filter_weights: np.ndarray = low_pass(0.25, 0.25, 4.0),
    phase_sigma: float = 1.0,
    time_sigma: float = 0.0,
    rng: np.random.Generator = np.random.default_rng(seed=None)
) -> np.ndarray:
    """Applies adjacent channel interference to the baseband data at a specified center frequency and power level. The 
    adjacent channel signal is a filtered, frequency-offset, randomly block time-shifted, randomly phase-perturbed 
    baseband copy that has similar bandwidth and modulation properties, but breaks phase coherence and time coherence 
    (the adjacent signal is decorrelated with original baseband data).

    Args:
        data (np.ndarray): Complex valued IQ data samples.
        sample_rate (float): Sampling rate (Fs). Default 4.0
        power (float): Adjacent interference signal power (linear, positive). Default 1.0 W (0 dBW).
        center_frequency (float): Adjacent interference signal center frequency (normalized relative to Fs). Default 0.2.
        filter_weights (np.ndarray): Lowpass filter weights applied to baseband signal data to band limit prior to creating
            adjacent signal. Default low_pass(0.25,0.25,4.0).
        phase_sigma (float): Standard deviation of Gaussian phase noise. Default 1.0.
        time_sigma (float): Standard deviation of Gaussian block time shift in samples. Default 0.0.
        rng (np.random.Generator, optional): Random number generator. Defaults to np.random.default_rng(seed=None).
    
    Returns:
        np.ndarray: Data with added adjacent interference.
    
    """
    N = len(data)
    t = np.arange(N) / sample_rate

    data_filtered = np.convolve(data, filter_weights)[-N:] # band limit original data
    phase_noise = rng.normal(0, phase_sigma, N)  # Gaussian phase noise
    interference = data_filtered * np.exp(1j*(2*np.pi*center_frequency*t + phase_noise)) # note: does not check aliasing

    time_shift = int(np.round(rng.normal(0, time_sigma, 1))[0]) # Gaussian block time shift for data (nearest sample)
    if time_shift > 0: # time shift with zero fill; # note: may produce discontinuities
        interference = np.roll(interference, time_shift) 
        interference[0:time_shift] = 0 + 1j*0
    elif time_shift < 0:
        interference = np.roll(interference, time_shift)
        interference[time_shift:0] = 0 + 1j*0

    # set interference power 
    est_power = np.sum(np.abs(interference)**2)/len(interference)
    interference = np.sqrt(power / est_power) * interference 

    return (data + interference).astype(torchsig_complex_data_type)

In [ ]:
rng = np.random.default_rng(seed=42)

N = 16384
sample_rate = 1.0
power = 0.5
center_frequency = 0.2
tone_freq = 0.042
filter_weights = low_pass(0.125, 0.125, sample_rate)
phase_sigma = 1.0
time_sigma = 0.0

tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
tone_data = tone_bb_data * np.exp(2j * np.pi * tone_freq * np.arange(N) / sample_rate) * np.sqrt(N) # f0 = 0.042, total power = 1.0 W
est_power = np.sum(np.abs(tone_data)**2)/len(tone_data)
print("tone_data power: ", est_power) # verify 1.0 W

data_adj0 = adjacent_channel_interference(
    data = tone_data,
    sample_rate = sample_rate,
    power = power,
    center_frequency = center_frequency,
    filter_weights = filter_weights,
    phase_sigma = phase_sigma,
    time_sigma = time_sigma,
    rng = rng
)
est_power = np.sum(np.abs(data_adj0)**2)/len(data_adj0)
print("total output power: ", est_power) # verify 1.0 W + 1.0 W = 2.0 W (uncorrelated)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(tone_data, Fs=sample_rate, scale='linear', sides='twosided', color='white');
spectrum, freqs, _ = ax.magnitude_spectrum(data_adj0, Fs=sample_rate, scale='linear', sides='twosided', color='red');
ax.set_yscale('log')
ax.set_xlabel('Frequency');
ax.set_ylabel('Magnitude [log]');


# verify peaks
D = np.abs(np.fft.fft(data_adj0, norm='ortho'))
freqs = np.fft.fftfreq(N) * sample_rate
peaks, _ = sp.find_peaks(D, height=1.0, distance=N/20)
top_two_indices = np.argsort(D[peaks])[-2:][::-1]
freqs0 = freqs[peaks[top_two_indices[0]]]
freqs1 = freqs[peaks[top_two_indices[1]]]
print("freqs[peaks] ", freqs[peaks])
print("D[peaks] ", D[peaks])
#print('top_two_indices', top_two_indices)
print('freqs[top_two_indices]', freqs[peaks[top_two_indices]])
print(freqs0, freqs1)

print(generate_tone_signal(num_iq_samples = N, scale = 1.0))

In [ ]:
rng = np.random.default_rng(seed=42)

tone_data = tone_bb_data * np.exp(2j * np.pi * -0.11 * np.arange(N) / sample_rate) * np.sqrt(N) # f0 = -0.11, total power = 1.0 W
sample_rate = 2.4

data_adj1 = adjacent_channel_interference(
    data = tone_data,
    power = 2.0,
    sample_rate = sample_rate,
    center_frequency = -0.05,
    filter_weights = low_pass(0.25, 0.25, 2.4),
    phase_sigma = 0.1,
    time_sigma = 10.0,
    rng = rng
)
est_power = np.sum(np.abs(data_adj1)**2)/len(data_adj1)
print("output power: ", 10**(est_power/10)) # verify 1.0 W + 1.0 W = 2.0 W (uncorrelated)

# plots
plt.style.use('dark_background')
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(tone_data, Fs=sample_rate, scale='linear', sides='twosided', color='white');
spectrum, freqs, _ = ax.magnitude_spectrum(data_adj1, Fs=sample_rate, scale='linear', sides='twosided', color='red');
ax.set_yscale('log')
ax.set_xlabel('Frequency');
ax.set_ylabel('Magnitude [log]');

In [ ]:
# verify peaks
D = np.abs(np.fft.fft(data_adj1, norm='ortho'))
freqs = np.fft.fftfreq(N) * sample_rate

peaks, _ = sp.find_peaks(D, height=0.5, distance=N/10)
print(freqs[peaks])
plt.plot(freqs, D)
plt.plot(freqs[peaks], D[peaks], "x")